# Inventory  Routing Problem: IRP

The two problems below are integrated  
- Inventory planning problem: Find the delivery date and quantity that minimizes inventory and out-of-stock costs.    
- Delivery planning problem: Find the delivery date and route that minimizes the total operating time of the delivery person. 

(What is inventory cost?) Cost required to keep inventory during a certain period of time.  
For example, location cost, management cost, etc. The higher the inventory holding ratio, the higher the cost.  
  
⇒ Only the necessary quantity should be kept in stock, and it is necessary to decide when and how much to deliver appropriately.

This notebook shows the implementtion of the following paper  
[Soysal+, 2015] Modeling an Inventory Routing Problem for perishable products with environmental considerations and demand uncertainty  
https://www.sciencedirect.com/science/article/abs/pii/S0925527315000766

In addition to the normal IRP, consider the following  factors
* Multiple periods of time  
* Perishable goods  
* Environment (CO2 emissions, fuel consumption)  
* Demand uncertainty (constraints are stochastic ⇒ stochastic constraint programming problem)  
  
Problem  
"Find the routes and volume of shipments in each period that minimizes the costs of inventory, destruction, fuel, and drivers."

Sensitivity analysis（Analyze optimal values by varying the following variables）  
* Demand：base, demand1, demand2  
* C=0.1(base), 0.05, 0.15, 0.2  
* Shelf life：m=2(base), 3, 4  
* Inventory holding costs (€/kg)：h=0.06(base), 0.03, 0.09, 0.12  
* Service level (processed rate) (%)：a=95(base), 90, 92.5, 97.5  
* Fuel (€/L)：l=1.7(base), 1.2, 2,2
* Vehicle speed(km/h)：f=80(base), 40, 120

In [1]:
import numpy as np
import math
import scipy as sp
from scipy import stats
import pulp
import time

## Definition of constants and variables

In [2]:
V = 12 # number of node(0:depot, 1~11:customer)
K = 7  # number of vehicles
T = 4  # duration

range_i = range(V)
range_j = range(V)
range_i_ = range(1,V)
range_j_ = range(1,V)
range_k = range(1,K+1)
range_t = range(1,T+1)

In [105]:
### Parameter 
# Table 4
xi = 1        # fuel-to-air mass ratio ξ
kappa = 44    # calorific value of fuel κ
psi = 737     # unit conversion Ψ 
k_e = 0.2     # engine friction coefficient
N_e = 33      # engine speed
V_e = 5       # engine displacement
rho = 1.2041  # air density ρ
A_e = 3.912   # surface area
mu = 6350     # weight on curve μ
g = 9.81      # acceleration of gravity
phi = 0       # road angle Φ
C_d = 0.7     # air resistance coefficient
C_r = 0.01    # rotational resistance coefficient
epsilon = 0.4 # drivetrain efficiency ε
varpi = 0.9   # Engine efficiency parameters ϖ
l = 1.7       # 1Lfuel cost per (€/L)
r = 0.003     # driver's wage per second(€/s)

# ↓
# Table 2
lambda_ = xi / (kappa*psi) 
y = k_e * N_e * V_e
gamma = 1 / (1000*epsilon*varpi)
beta = 0.5 * C_d * A_e * rho
s_ = g * np.sin(phi) + g * C_r * np.cos(phi)

# 6.1 Description and data
c = 10000     # vehicle capacity (10 tonnes = 10000 kg)
b = 0.21      # model M, M_P Parameters needed to calculate fuel costs at (L/km) (4.1reference)
u = 2.63      # fuel conversion factor (kg/L)
f = 80        # vehicle speed (km/h)
C = 0.1       # restrict(21)
alpha = 0.95      # service level (%)
h = 0.06      # inventory holding cost (€/kg)
m = 2         # shelf life (week)
p = 0.6       # Corruption cost (€/kg)

# 3.3
# standard normal random variate with cumulative probability (lower probability) alpha
Za = stats.norm.ppf(loc=0, scale=1, q=alpha)

In [106]:
## demand (4weeks)
# demand means 
# Add 0 for subscript．1~4 store the value in
d_mu = [
    [0,0,0,0,0],
    [0,900,400,1000,600],
    [0,1400,1200,17,1200],
    [0,500,500,1250,600],
    [0,1100,2500,500,400],
    [0,1050,900,1500,1100],
    [0,1200,500,400,1400],
    [0,800,700,500,500],
    [0,1900,400,300,1300],
    [0,800,400,700,1300],
    [0,1100,1600,400,300],
    [0,2600,3200,2500,3200],
]

# calculated for constraint (21)
right_of_const_21 = np.zeros([V, T+1])
for t in range_t:
    for i in range_i_:
        right_of_const_21[i][t] = sum(d_mu[i][s] for s in range(1, t+1)) + np.sqrt(sum(d_mu[i][s]**2 for s in range(1, t+1)))*C*Za

## distance (12*12)
a = [
    [0,67,89.2,126,78.1,70.6,106,66.3,64.4,156,151,35.5],
    [73.2,0,154,191,143,144,141,101,74.3,166,176,61.5],
    [70.8,136,0,65.9,62.9,113,158,118,126,218,212,97.2],
    [126,192,69.5,0,98.9,171,233,193,182,274,268,153],
    [78.4,144,63.2,99.7,0,123,185,145,134,226,220,105],
    [70.9,144,105,163,115,0,50.2,58.4,120,155,220,105],
    [106,131,161,222,175,50.9,0,41.6,75.3,105,199,84.4],
    [66.5,91.2,121,182,135,58.2,40.1,0,35.3,117,159,44.4],
    [67.4,74.9,149,185,137,92.7,74.4,34.5,0,92.1,120,34.8],
    [158,166,239,276,228,155,106,116,92.4,0,69.6,126],
    [150,176,232,268,220,221,192,152,119,70,0,119],
    [35,60.3,116,153,105,106,83.6,43.7,30.6,123,118,0]
]

## Formulation

In [107]:
problem = pulp.LpProblem("IRP", pulp.LpMinimize)

### Defining decision variables

In [108]:
### decision variables
# X <-(11)
# Binary variable equal to 1 if vehicle k goes from i to j in period t, and 0 otherwise
# 0-1 variable (path)
X = pulp.LpVariable.dicts("X", (range_i, range_j, range_k, range_t), cat="Binary")
for t in range_t:
    for k in range_k:
        for j in range_j:
            for i in range_i:
                if i == j:
                    problem += X[i][j][k][t] == 0

# F <-(12)
# The load on vehicle k which goes from i to j in period t, kg
# load
F = pulp.LpVariable.dicts("F", (range_i, range_j, range_k, range_t), lowBound=0)
for t in range_t:
    for k in range_k:
        for j in range_j:
            for i in range_i:
                if i == j:
                    problem += F[i][j][k][t] == 0

### Definition of data required for objective function and constraints

In [109]:
# I <-(13)
# The amount of inventory at customer i at the end of period t, where I_i,0 = 0
# In stock
I = pulp.LpVariable.dicts("I",(range_i_, range_t))

# I+ <-(14)
# Derived decision variable to calculate positive inventory levels, kg
# Value obtained from the constraint of the expected value of the inventory amount (inventory amount)
Ip = pulp.LpVariable.dicts("Ip", (range_i_, range_t),lowBound =0)

# W <-(14)
# The amount of waste at customer i at the end of period t, kg
# Amount of corruption
W = pulp.LpVariable.dicts("W", (range_i_, range_t),lowBound =0)

# Q <-(15)
# The amount of product delivered by vehicle k to customer i in the beginning of period t, kg
# # the amount the customer receives
Q = pulp.LpVariable.dicts("Q", (range_i_, range_k, range_t) ,lowBound =0)

### Objective function definition

In [110]:
### objective function
# (1)
inventory_cost = pulp.lpSum(Ip[i][t] * h for i in range_i_ for t in range_t)

waste_cost = pulp.lpSum(W[i][t] * p for i in range_i_ for t in range(m,T+1))

fuel_cost = pulp.lpSum(lambda_*(y*(a[i][j])*X[i][j][k][t] + gamma*beta*a[i][j]*f**2*X[i][j][k][t]
                               + gamma*s_*(mu*X[i][j][k][t]+F[i][j][k][t])*a[i][j]) * l if i != j else 0
                          for t in range_t for k in range_k for j in range_j for i in range_i)

driver_cost = pulp.lpSum((a[i][j] / f) * X[i][j][k][t] * r if i != j else 0
                          for t in range_t for k in range_k for j in range_j for i in range_i)

problem += inventory_cost + waste_cost + fuel_cost + driver_cost

### Constraint definition

In [111]:
### constraints
## related to inventory
# (2)
# Expected value of inventory level for each customer in each period
for t in range_t:
    for i in range_i_:
        problem += I[i][t] == pulp.lpSum(Q[i][k][s] for s in range(1,t+1) for k in range_k) \
        - pulp.lpSum(d_mu[i][s] + W[i][s] for s in range(1,t))
        
# (3)
# Defining the variable Ip used to calculate the inventory cost for the objective function
for t in range_t:
    for i in range_i_:
        problem += Ip[i][t] >= I[i][t]

# (4), (5)
# Expected spoilage amount for each customer in each period
for t in range_t:
    if t >= m:
        for i in range_i_:
            problem += W[i][t] >= I[i][t-m+1] - pulp.lpSum(d_mu[i][a] for a in range(t-m+2, t+1)) \
            - pulp.lpSum(W[i][a] for a in range(t-m+2, t))
    else:
        for i in range_i_:
            problem += W[i][t] == 0

# (6)->(21)

## related to routing
# (7)
# The number of vehicles coming to a customer is the same as the number of vehicles leaving
for t in range_t:
    for k in range_k:
        for j in range_j_:
            left = pulp.lpSum(X[i][j][k][t] if i != j else 0 for i in range_i)
            right = pulp.lpSum(X[j][i][k][t] if i != j else 0 for i in range_i)
            problem +=  left == right

# (8)
# Each customer location is visited no more than once by one vehicle
for t in range_t:
    for k in range_k:
        for i in range_i:
            problem += pulp.lpSum(X[i][j][k][t] if i != j else 0 for j in range_j) <= 1

# (9)
# The amount of cargo is reduced by the amount delivered at the customer's site.
for t in range_t:
    for k in range_k:
        for i in range_i_:
            left = pulp.lpSum(F[i][j][k][t] if i != j else 0 for j in range_j)
            right = pulp.lpSum(F[j][i][k][t] if i != j else 0 for j in range_j) - Q[i][k][t]
            problem += left == right

# (10)
# capacity limit
for t in range_t:
    for k in range_k:
        for i in range_i:
            for j in range_j:
                if i != j:
                    left = F[i][j][k][t]
                    right = c * X[j][i][k][t]
                    problem += left <= right       
                    
# (21)
# Service level constraints on out-of-stock probabilities at the end of each period
for t in range_t:
    for i in range_i_:
        left = pulp.lpSum(Q[i][k][s] for s in range(1,t+1) for k in range_k) \
        - pulp.lpSum(W[i][s] for s in range(1,t))
        problem += left >= right_of_const_21[i][t]

# Run optimization

In [ ]:
Run optimization### run
from pulp import PULP_CBC_CMD
t_start = time.time() 
result_status = problem.solve(solver=PULP_CBC_CMD())
t_end = time.time() 

# Solution verification

In [ ]:
print("optimality = {:}, objective function value = {:}, calculating time = {:} (Second)"
      .format(pulp.LpStatus[result_status], pulp.value(problem.objective), t_end - t_start))

# Optimized solution

In [100]:
print("unit:€")
print("inventory_cost:",pulp.value(inventory_cost))
print("waste_cost",pulp.value(waste_cost))
print("fuel_cost",pulp.value(fuel_cost))
print("driver_cost",pulp.value(driver_cost))
print("cost:", pulp.value(problem.objective))

unit:€
inventory_cost: 610.3413473999999
waste_cost 696.3354126
fuel_cost 3.0506674324427734
driver_cost 0.03375
cost: 1309.7611774324428
